In [13]:
suppressWarnings(
  suppressPackageStartupMessages(
    {
      library(readxl)
      library(writexl)
      library(dplyr)
      library(purrr)
      library(tibble)
      library(devtools)
    }
  )
)

# Installs from the local folder (your modified clone)
# if (requireNamespace("idiolect", quietly = TRUE)) {
#   message("Removing Installed Version of Idiolect")
#   invisible(capture.output(remove.packages("idiolect")))
# }

message("Installing CRAN Version of Idiolect")
devtools::install("/Users/user/Documents/GitHub/idiolect", upgrade = "never")

library(idiolect)

Installing CRAN Version of Idiolect
── R CMD build ─────────────────────────────────────────────────────────────────
✔  checking for file ‘/Users/user/Documents/GitHub/idiolect/DESCRIPTION’ (596ms)
─  preparing ‘idiolect’:
✔  checking DESCRIPTION meta-information ...
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘idiolect_1.1.1.9000.tar.gz’
   
Running /Library/Frameworks/R.framework/Resources/bin/R CMD INSTALL \
  /var/folders/xx/hy496x3x5sn4hy9gy1fk19lw0000gp/T//RtmpA0Of8b/idiolect_1.1.1.9000.tar.gz \
  --install-tests 
* installing to library ‘/Library/Frameworks/R.framework/Versions/4.2-arm64/Resources/library’
* installing *source* package ‘idiolect’ ...
** using staged installation
** R
** data
*** moving datasets to lazyload DB
** inst
** tests
** byte-compile and prepare package for lazy loading
Warning messages:
1: package ‘quanteda’ was built under R version 4.2.3 
2: In .recacheSubclasses(

In [14]:
base_loc <- '/Volumes/BCross/av_datasets_experiments/Baseline Results'

n_gram_data <- readRDS(paste0(base_loc, "/n_gram_tracing_results.rds"))

In [16]:
distinct_datasets <- n_gram_data |>
  select(data_type, corpus, coefficient, n) |>
  distinct()

In [17]:
test_distinct <- distinct_datasets |> filter(data_type == 'test')
training_distinct <- distinct_datasets |> filter(data_type == 'training')

In [18]:
calculate_single_distinct <- function(score_df, problem_df){

  results_list <- vector("list", nrow(problem_df))
  n_total <- nrow(problem_df)

  for (i in seq_len(n_total)) {
    message("Processing row ", i, " out of ", n_total)

    filtered_problem <- problem_df[i, ]

    d_type <- filtered_problem |> dplyr::pull(data_type) |> as.character()
    corp   <- filtered_problem |> dplyr::pull(corpus) |> as.character()
    coef   <- filtered_problem |> dplyr::pull(coefficient) |> as.character()
    n_val  <- filtered_problem |> dplyr::pull(n) |> as.integer()

    filtered_score <- score_df |>
      dplyr::filter(
        data_type == d_type,
        corpus == corp,
        coefficient == coef,
        n == n_val
      )

    if (nrow(filtered_score) == 0) {
      next
    }

    res <- performance(filtered_score)
    perf <- res$evaluation

    perf <- perf |>
      dplyr::mutate(
        data_type = d_type,
        corpus = corp,
        coefficient = coef,
        n = n_val
      ) |>
      dplyr::relocate(data_type, corpus, coefficient, n)

    results_list[[i]] <- perf
  }

  dplyr::bind_rows(results_list)
}

In [ ]:
# training_results <- calculate_single_distinct(n_gram_data, training_distinct)
# saveRDS(training_results, paste0(base_loc, "/training/n_gram_tracing_performance.rds"))

Processing row 1 out of 180
  |======================================================================| 100%
Processing row 2 out of 180
  |======================================================================| 100%
Processing row 3 out of 180
  |======================================================================| 100%
Processing row 4 out of 180
  |======================================================================| 100%
Processing row 5 out of 180
  |======================================================================| 100%
Processing row 6 out of 180
  |======================================================================| 100%
Processing row 7 out of 180
  |======================================================================| 100%
Processing row 8 out of 180
  |======================================================================| 100%
Processing row 9 out of 180
  |======================================================================| 100%
Processing row 10 out of 180

Warning messages:
1: In predict.lm(object, newdata, se.fit, scale = 1, type = if (type ==  :
  prediction from a rank-deficient fit may be misleading
2: In predict.lm(object, newdata, se.fit, scale = 1, type = if (type ==  :
  prediction from a rank-deficient fit may be misleading
3: In predict.lm(object, newdata, se.fit, scale = 1, type = if (type ==  :
  prediction from a rank-deficient fit may be misleading
4: In predict.lm(object, newdata, se.fit, scale = 1, type = if (type ==  :
  prediction from a rank-deficient fit may be misleading
5: In predict.lm(object, newdata, se.fit, scale = 1, type = if (type ==  :
  prediction from a rank-deficient fit may be misleading
6: In confusionMatrix.default(as.factor(res.res$predicted), as.factor(res.res$target),  :
  Levels are not in the same order for reference and data. Refactoring data to match.


In [ ]:
# test_results <- calculate_single_distinct(n_gram_data, test_distinct)
# saveRDS(test_results, paste0(base_loc, "/test/n_gram_tracing_performance.rds"))

Processing row 1 out of 180
  |======================================================================| 100%
Processing row 2 out of 180
  |======================================================================| 100%
Processing row 3 out of 180
  |======================================================================| 100%
Processing row 4 out of 180
  |======================================================================| 100%
Processing row 5 out of 180
  |======================================================================| 100%
Processing row 6 out of 180
  |======================================================================| 100%
Processing row 7 out of 180
  |======================================================================| 100%
Processing row 8 out of 180
  |======================================================================| 100%
Processing row 9 out of 180
  |======================================================================| 100%
Processing row 10 out of 180

There were 50 or more warnings (use warnings() to see the first 50)


In [25]:
calculate_calibrated_distinct <- function(score_df, problem_df){

  training_data <- score_df |> filter(data_type == 'training')
  test_data <- score_df |> filter(data_type == 'test')

  training_problems <- problem_df |> filter(data_type == 'training') |> select(-data_type)
  test_problems <- problem_df |> filter(data_type == 'test') |> select(-data_type)

  combined_problems <- test_problems |>
    inner_join(training_problems, by=c('corpus', 'coefficient', 'n'))

  results_list <- vector("list", nrow(combined_problems))
  n_total <- nrow(combined_problems)

  for (i in seq_len(n_total)) {
    message("Processing row ", i, " out of ", n_total)

    filtered_problem <- problem_df[i, ]

    corp   <- filtered_problem |> dplyr::pull(corpus) |> as.character()
    coef   <- filtered_problem |> dplyr::pull(coefficient) |> as.character()
    n_val  <- filtered_problem |> dplyr::pull(n) |> as.integer()

    filtered_training <- training_data |>
      dplyr::filter(
        corpus == corp,
        coefficient == coef,
        n == n_val
      )

    filtered_test <- test_data |>
      dplyr::filter(
        corpus == corp,
        coefficient == coef,
        n == n_val
      )

    res <- performance(filtered_test, training = filtered_training)
    perf <- res$evaluation

    perf <- perf |>
      dplyr::mutate(
        corpus = corp,
        coefficient = coef,
        n = n_val
      ) |>
      dplyr::relocate(corpus, coefficient, n)

    results_list[[i]] <- perf
  }

  dplyr::bind_rows(results_list)
}

In [ ]:
# combined_data <- calculate_calibrated_distinct(n_gram_data, distinct_datasets)
# saveRDS(combined_data, paste0(base_loc, "/n_gram_tracing_calibrated_performance.rds"))

Processing row 1 out of 180
Processing row 2 out of 180
Processing row 3 out of 180
Processing row 4 out of 180
Processing row 5 out of 180
Processing row 6 out of 180
Processing row 7 out of 180
Processing row 8 out of 180
Processing row 9 out of 180
Processing row 10 out of 180
Processing row 11 out of 180
Processing row 12 out of 180
Processing row 13 out of 180
Processing row 14 out of 180
Processing row 15 out of 180
Processing row 16 out of 180
Processing row 17 out of 180
Processing row 18 out of 180
Processing row 19 out of 180
Processing row 20 out of 180
Processing row 21 out of 180
Processing row 22 out of 180
Processing row 23 out of 180
Processing row 24 out of 180
Processing row 25 out of 180
Processing row 26 out of 180
Processing row 27 out of 180
Processing row 28 out of 180
Processing row 29 out of 180
Processing row 30 out of 180
Processing row 31 out of 180
Processing row 32 out of 180
Processing row 33 out of 180
Processing row 34 out of 180
Processing row 35 out o

Warning messages:
1: In confusionMatrix.default(as.factor(res.res$predicted), as.factor(res.res$target),  :
  Levels are not in the same order for reference and data. Refactoring data to match.
2: In confusionMatrix.default(as.factor(res.res$predicted), as.factor(res.res$target),  :
  Levels are not in the same order for reference and data. Refactoring data to match.
3: In confusionMatrix.default(as.factor(res.res$predicted), as.factor(res.res$target),  :
  Levels are not in the same order for reference and data. Refactoring data to match.
4: In confusionMatrix.default(as.factor(res.res$predicted), as.factor(res.res$target),  :
  Levels are not in the same order for reference and data. Refactoring data to match.
5: In confusionMatrix.default(as.factor(res.res$predicted), as.factor(res.res$target),  :
  Levels are not in the same order for reference and data. Refactoring data to match.
6: In confusionMatrix.default(as.factor(res.res$predicted), as.factor(res.res$target),  :
  Levels are

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import ceil

def _order_n_cats(cats):
    """Sort numeric n ascending; put non-numeric after; 'None' last."""
    def key(x):
        x = str(x)
        if x.lower() == "none":
            return (2, float("inf"))
        try:
            return (0, float(x))
        except ValueError:
            return (1, x)
    return sorted(pd.unique([str(c) for c in cats]), key=key)

def facet_metric_vs_n_by_corpus(
    df,
    *,
    x_col="n",
    y_col="balanced_accuracy",
    facet_col="corpus",
    group_col="coefficient",
    agg_fn="mean",
    ncols=3,
    sharey=True,
    title=None,
    legend=True,
):
    """
    Facet wrap by corpus (one subplot per corpus).
    X: n (as categorical: "2", "3", ...; missing -> "None")
    Y: y_col (e.g., balanced_accuracy)
    Lines: coefficient
    """
    d = df.copy()

    # Required columns
    for col in [x_col, y_col, facet_col, group_col]:
        if col not in d.columns:
            raise ValueError(f"Missing '{col}' in df.")

    # Build x categorical labels from x_col
    def _to_n_cat(v):
        if pd.isna(v):
            return "None"
        try:
            fv = float(v)
            return str(int(fv)) if fv.is_integer() else str(fv)
        except Exception:
            return str(v)

    d["n_cat"] = d[x_col].map(_to_n_cat)

    # Aggregate in case multiple rows per (corpus, coefficient, n)
    grp = d.groupby([facet_col, group_col, "n_cat"], dropna=False)[y_col]
    if agg_fn == "mean":
        plot_df = grp.mean().reset_index()
    elif agg_fn == "median":
        plot_df = grp.median().reset_index()
    else:
        raise ValueError("agg_fn must be 'mean' or 'median'")

    # Global ordered x-axis across all facets
    x_order = _order_n_cats(plot_df["n_cat"])
    x_pos = np.arange(len(x_order))
    x_index = {lab: i for i, lab in enumerate(x_order)}

    # Facets = corpora
    facet_vals = sorted(plot_df[facet_col].dropna().unique())
    n_facets = len(facet_vals)
    nrows = ceil(n_facets / ncols)

    fig, axes = plt.subplots(
        nrows=nrows,
        ncols=ncols,
        figsize=(4.8 * ncols, 3.6 * nrows),
        sharex=True,
        sharey=sharey,
    )
    axes = np.array(axes).reshape(-1)

    legend_handles = None
    legend_labels = None

    for i, fac in enumerate(facet_vals):
        ax = axes[i]
        sub_fac = plot_df[plot_df[facet_col] == fac]

        for g in sorted(sub_fac[group_col].dropna().unique()):
            sub_g = sub_fac[sub_fac[group_col] == g]

            y = np.full(len(x_order), np.nan, dtype=float)
            for _, r in sub_g.iterrows():
                y[x_index[r["n_cat"]]] = float(r[y_col])

            ax.plot(x_pos, y, marker="o", linewidth=2, label=f"{group_col}={g}")

        ax.set_title(str(fac))
        ax.grid(True, alpha=0.3)
        ax.set_ylabel(y_col)

        if legend and legend_handles is None:
            legend_handles, legend_labels = ax.get_legend_handles_labels()

    # Turn off unused axes
    for j in range(n_facets, len(axes)):
        axes[j].axis("off")

    # X ticks/labels
    for ax in axes[:n_facets]:
        ax.set_xticks(x_pos)
        ax.set_xticklabels(x_order)

    # X label only on bottom row
    for ax in axes[(nrows - 1) * ncols : nrows * ncols]:
        if ax.has_data():
            ax.set_xlabel(x_col)

    # Shared legend
    if legend and legend_handles:
        fig.legend(
            legend_handles,
            legend_labels,
            loc="upper center",
            bbox_to_anchor=(0.5, 1.02),
            ncol=min(len(legend_labels), 6),
            frameon=False,
        )

    if title:
        fig.suptitle(title, y=1.08 if legend else 1.02)

    plt.tight_layout()
    return fig, axes[:n_facets]

# Example:
# fig, axes = facet_metric_vs_n_by_corpus(df, y_col="balanced_accuracy", title="Balanced accuracy vs n")
# plt.show()

In [1]:
df = readRDS('/Volumes/BCross/av_datasets_experiments/Baseline Results/n_gram_tracing_calibrated_performance.rds')